# Evaluate Fisher information

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os

import madminer.core
from madminer.fisherinformation import FisherInformation, profile_information, project_information
from madminer.plotting import plot_fisher_information_contours_2d


In [2]:
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/'

In [4]:
sample_dir = base_dir + 'data/samples/wgamma_sys/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma_sys/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma_sys/'
result_dir = base_dir + 'data/results/wgamma_sys/'

## FisherInformation instances

In [5]:
fisher_all = FisherInformation(sample_dir + 'samples.h5')
fisher_tight = FisherInformation(sample_dir + 'samples_tight.h5')

18:27 madminer.fisherinfor INFO    Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples.h5
18:27 madminer.fisherinfor INFO    Found 2 parameters
18:27 madminer.fisherinfor WARNING Did not find nuisance parameters!
18:27 madminer.fisherinfor INFO    Found 103 benchmarks, of which 6 physical
18:27 madminer.fisherinfor INFO    Found 33 observables: et_miss, phi_miss, e_visible, eta_visible, e_l1, pt_l1, eta_l1, phi_l1, e_a1, pt_a1, eta_a1, phi_a1, e_j1, pt_j1, eta_j1, phi_j1, pdgid_l1, m_la, m_lmet, m_amet, pt_la, pt_lmet, pt_amet, deltaphi_la, deltaphi_lmet, deltaphi_amet, deltaeta_la, m_almet, pt_almet, mt, phi_minus, phi_plus, phi
18:27 madminer.fisherinfor INFO    Found 587119 events
18:27 madminer.fisherinfor INFO    Found morphing setup with 6 components
18:27 madminer.fisherinfor INFO    Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples_tight.h5
18:27 madminer.fisherin

## Profiling

In [11]:
def profile(info, cov):
    fisher_info_nuisance = fisher_tight.calculate_fisher_information_nuisance_constraints()
    return profile_information(info + fisher_info_nuisance, [0,1], covariance=cov)
    

## Truth-level info

In [6]:
fisher_info, cov = fisher_all.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_parton.npy', fisher_info)
np.save(result_dir + 'information_covariance_parton.npy', cov)


In [ ]:
fisher_info, cov = fisher_tight.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_parton_tight.npy', fisher_info)
np.save(result_dir + 'information_covariance_parton_tight.npy', cov)

fisher_info, cov = profile(fisher_info, cov)

np.save(result_dir + 'profiled_information_parton_tight.npy', fisher_info)
np.save(result_dir + 'profiled_information_covariance_parton_tight.npy', cov)


## Rate-only info

In [ ]:
fisher_info, cov = fisher_all.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_xsec.npy', fisher_info)
np.save(result_dir + 'information_covariance_xsec.npy', cov)

fisher_info, cov = profile(fisher_info, cov)

np.save(result_dir + 'profiled_information_xsec.npy', fisher_info)
np.save(result_dir + 'profiled_information_covariance_xsec.npy', cov)


In [7]:
fisher_info, cov = fisher_tight.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_xsec_tight.npy', fisher_info)
np.save(result_dir + 'information_covariance_xsec_tight.npy', cov)

fisher_info, cov = profile(fisher_info, cov)

np.save(result_dir + 'profiled_information_xsec_tight.npy', fisher_info)
np.save(result_dir + 'profiled_information_covariance_xsec_tight.npy', cov)


## Full Fisher info (ML)

In [ ]:
mean, cov = fisher_all.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all',
    mode="score"
)

np.save(result_dir + 'information_full.npy', mean)
np.save(result_dir + 'information_covariance_full.npy', cov)

In [ ]:
mean, cov = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all_tight',
    mode="score",
    batch_size=100
)

np.save(result_dir + 'information_full_tight.npy', mean)
np.save(result_dir + 'information_covariance_full_tight.npy', cov)

fisher_info, cov = profile(fisher_info, cov)

np.save(result_dir + 'profiled_information_full_tight.npy', fisher_info)
np.save(result_dir + 'profiled_information_covariance_full_tight.npy', cov)



In [7]:
fisher_info, cov = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all_tight',
    mode="information",
)

np.save(result_dir + 'information_full_tight_infomode.npy', fisher_info)
np.save(result_dir + 'information_covariance_full_tight_infomode.npy', cov)

fisher_info, cov = profile(fisher_info, cov)

np.save(result_dir + 'profiled_information_full_tight_infomode.npy', fisher_info)
np.save(result_dir + 'profiled_information_covariance_full_tight_infomode.npy', cov)


18:27 madminer.ml          INFO    Found ensemble with 3 estimators and expectations [[-7.48849735e-02  1.42248616e-01 -1.31866600e-05 -1.02201756e-03
   1.28867696e-05 -7.95546977e-04  6.15302182e-04  1.04787434e-03
  -1.09314802e-03 -2.08686804e-03  1.59275834e-03 -1.07890961e-03
   9.52444621e-04 -2.03196099e-03  1.68410095e-03 -3.38292040e-04
  -1.05431501e-03  1.71077973e-03 -2.93941074e-03 -1.59854966e-03
   7.85581826e-04  2.13778089e-03 -2.41466612e-03  1.87148992e-03
  -1.42234669e-03  4.95984023e-05 -1.90590182e-03 -8.47952673e-04
  -9.14361735e-05  1.97054096e-03 -8.79976025e-04  1.45764425e-04
  -1.78752874e-04 -1.41718914e-03 -1.10227316e-04 -2.12507392e-03
   1.41736120e-03  1.11997698e-03  4.01392526e-06 -3.33469798e-04
  -7.02715362e-04 -1.72730489e-03  4.99627087e-04 -7.24445039e-04
  -4.85265278e-04 -1.69891026e-03  1.72403501e-03  7.43082637e-05
  -1.70557690e-03  5.17499517e-04 -2.61418638e-04 -4.85865108e-04
  -1.51122699e-03 -1.01623125e-03  1.18056219e-03 -7.9552

18:28 madminer.ml          INFO    Expected per-event score (should be close to zero): [-4.24816799e+00 -1.05508595e+01  1.73758744e-05 -2.08255071e-02
  1.17431198e-04 -2.46108016e-07  1.31894398e-04  3.44787072e-03
 -3.52456421e-03 -4.96103847e-03  4.46816301e-03 -2.58761435e-03
  2.75447499e-03 -5.81192877e-03  6.65699830e-03  1.72851689e-03
 -1.29235652e-03  9.81470011e-03 -1.37727447e-02 -2.71262578e-03
  1.00094214e-04  3.46344686e-03 -4.95457929e-03 -7.17868423e-03
  1.57088984e-03  2.57485453e-02 -2.69884132e-02 -7.76521582e-03
  1.13408212e-02  6.20904239e-03 -1.50130531e-02 -3.12567921e-03
  2.62534525e-03 -3.49324173e-03 -1.01550319e-03 -8.37001111e-03
  4.91511822e-03  2.26348825e-03  1.39123620e-03 -8.56107567e-03
  4.84335888e-03 -1.16227260e-02  8.38907436e-03  1.36133167e-03
  4.17867908e-04 -4.00841888e-03  3.56349838e-03 -3.20327538e-03
 -7.83914758e-04 -2.74374266e-03  4.75632632e-03  4.05367144e-04
  1.11433575e-02 -1.31780095e-03  5.19838475e-04  9.54293238e-04
 -1

NameError: name 'fisher_info' is not defined

In [9]:
project_information(mean,[0,1])

array([[1228962.86792385,   -2967.65113762],
       [  -2967.65113762,   72341.65035628]])

In [12]:
fisher_info, cov = profile(mean, cov)
fisher_info

array([[1224487.08306822,   -3007.691043  ],
       [  -3007.691043  ,   72270.63835077]])

In [ ]:
mean, cov = fisher_all.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all_shuffled',
    mode="score"
)

np.save(result_dir + 'information_full_shuffled.npy', mean)
np.save(result_dir + 'information_covariance_full_shuffled.npy', cov)

## Minimal observable set (ML)

In [ ]:
mean, cov = fisher_all.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_minimal',
    mode="score"
)

np.save(result_dir + 'information_minimal.npy', mean)
np.save(result_dir + 'information_covariance_minimal.npy', cov)

In [ ]:
mean, cov = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_minimal_tight',
    mode="score"
)

np.save(result_dir + 'information_minimal_tight.npy', mean)
np.save(result_dir + 'information_covariance_minimal_tight.npy', cov)

## Info in observables (ML)

In [ ]:
mean, cov = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_phi_tight',
    mode="score"
)

np.save(result_dir + 'information_phi_tight.npy', mean)
np.save(result_dir + 'information_covariance_phi_tight.npy', cov)


## Histogram info

In [ ]:
filenames = ['phi', 'met', 'ptl', 'pta', 'deltaphi_lv', 'deltaphi_la']
observables = ['phi', 'et_miss', 'pt_l1', 'pt_a1', 'deltaphi_lmet', 'deltaphi_la']
bins = 25

In [ ]:
for filename, obs in zip(filenames, observables):
    print(filename)
    info, cov = fisher_tight.calculate_fisher_information_hist1d(
        theta=[0.,0.],
        luminosity=300000.,
        observable=obs,
        nbins=bins,
        histrange=None
    )
    
    np.save(result_dir + 'information_histo_{}_tight.npy'.format(filename), info)
    np.save(result_dir + 'information_covariance_histo_{}_tight.npy'.format(filename), cov)
    